In [6]:
from bs4 import BeautifulSoup

import requests
import re

import pandas as pd

from geopy.geocoders import Photon
from geopy.geocoders import Nominatim
import unidecode

In [2]:
# Reemplaza 'ruta_al_archivo.csv' con la ruta completa al archivo CSV que deseas leer
ruta_al_archivo = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Mapa_Casillas_2006_Reconstruido/Puntos de Internacion Maritima/puntos de internacion maritima.csv'
df = pd.read_csv(ruta_al_archivo)



             puerto                  edo  ciudad      ruta coordenadas
0          Rosarito      Baja California     NaN  pacifico         NaN
1         El Sauzal      Baja California     NaN  pacifico         NaN
2  Puerto Escondido      Baja California     NaN  pacifico         NaN
3    Puerto Peñasco               Sonora     NaN  pacifico         NaN
4    Isla de Cedros  Baja California Sur     NaN  pacifico         NaN


In [7]:
#Quitamos acentos de todos lados
def quitar_acentos(valor):
    """Función para quitar acentos de una cadena de texto."""
    if isinstance(valor, str):  # Solo aplicar a cadenas de texto
        return unidecode.unidecode(valor)  # Corrección aquí
    return valor

# Aplicamos la función quitar_acentos a todas las columnas del DataFrame
for columna in df.columns:
    df[columna] = df[columna].apply(quitar_acentos)

In [13]:
# Combinamos las columnas 'puerto' y 'edo' en una nueva columna 'ciudad'
df['ciudad'] = df['puerto'] + ', ' + df['edo']
df = df.drop('coordenadas', axis=1)

In [14]:
df.head()

,puerto,edo,ciudad,ruta,latitud,longitud
0,Rosarito,Baja California,"Rosarito, Baja California",pacifico,32.365166,-117.055792
1,El Sauzal,Baja California,"El Sauzal, Baja California",pacifico,31.893300,-116.691263
2,Puerto Escondido,Baja California,"Puerto Escondido, Baja California",pacifico,25.811704,-111.308949
3,Puerto Penasco,Sonora,"Puerto Penasco, Sonora",pacifico,31.317087,-113.531819
4,Isla de Cedros,Baja California Sur,"Isla de Cedros, Baja California Sur",pacifico,24.068716,-110.304739


### get lat long

In [11]:
# Inicializar el geolocalizador de Photon
geolocator = Photon(user_agent="measurement", timeout=10)

# Definir la función para obtener latitud y longitud
def get_lat_lon(address):
    try:
        location = geolocator.geocode(address)
        if location:  # Verificar si se encontró la ubicación
            return location.latitude, location.longitude
        else:
            return '', ''  # Devolver cadenas vacías si no se encuentra la ubicación
    except AttributeError:
        return '', ''

#Aplicar la función a la columna 'Ciudad' y dividir los resultados en las columnas 'Latitud' y 'Longitud'
df[['latitud', 'longitud']] = df['ciudad'].apply(lambda x: pd.Series(get_lat_lon(x)))

### actualizar archivo


In [15]:
# Especificamos la ruta donde queremos guardar el archivo CSV
ruta_archivo1 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Mapa_Casillas_2006_Reconstruido/Puntos de Internacion Maritima/puntos_de_internacion_maritima_1.csv'

# Guardamos el DataFrame en el archivo CSV en la ruta especificada
df.to_csv(ruta_archivo1, index=False)
